In [296]:
from sys import path

import numpy as np
import pandas as pd 
from scipy.stats import norm
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

%load_ext autoreload
%autoreload 2

# user-written 
import w8_estimation as est 
import w8_LinearModel as lm
import w8_probit as probit
import w8_logit as logit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [297]:
# Outcome label
y_lab = 'anyuseofforce_coded'

# Dataset columns
rawdat_columns = [
    'anyuseofforce_coded',
    
    # Subject (civilian) characteristics
    'sblack',
    'shisp',
    'swhite',
    'smale',
    'sother', 
    'sage',
    'sempl', 
    'sincome',
    'spop', 
    'sbehavior',
    
    # Officer characteristics
    'omajblack',
    'omajhisp',
    'omajwhite',
    'omajother', 
    'osplit', 
    
    # Encounter characteristics
    'daytime',
    'inctype_lin', 
    'year'
]


In [298]:
dat = pd.read_csv('ppcs_cc.csv')

N = dat.shape[0]

# reorder columns 
dat = dat[[y_lab] + x_lab].copy()

dat.head(5)

assert dat.notnull().all(axis=1).all(), 'Missings in the dataset, take them out!'

In [299]:
# Final X-matrix variable labels 
x_lab = [
    # Subject vars (white is reference)
    'sblack',
    'shisp',
    'smale',
    'sage',

    # Officer vars (white is reference)
    #'omajblack', --> Udelukkes fordi der ikke er nogen
    'omajhisp',
    #'omajother',

    # Encounter vars
    'daytime'
]

In [300]:
y = dat[y_lab].values
x = dat[x_lab].values
K = x.shape[1]

print("Shape x:", x.shape)
print("Rank x:", np.linalg.matrix_rank(x))
y.shape

Shape x: (3799, 6)
Rank x: 6


(3799,)

In [301]:
# OLS estimates
ols_results = lm.estimate(y, x, robust_se=True)
ols_tab = lm.print_table((y_lab, x_lab), ols_results, title='LPM results')
ols_tab

LPM results
Dependent variable: anyuseofforce_coded

R2 = 0.004
sigma2 = nan


,b_hat,se,t
sblack,0.0048,0.0042,1.1465
shisp,0.0126,0.0063,1.9881
smale,0.0068,0.0022,3.0771
sage,-0.0000,0.0000,-0.4581
omajhisp,0.0042,0.0107,0.3885
daytime,-0.0010,0.0022,-0.4525


In [302]:
theta0 = probit.starting_values(y, x)
theta0.ndim==1

True

In [303]:
ll = probit.loglikelihood(theta0, y, x)
np.isclose(np.mean(ll), -1.0411283428047824)

np.False_

In [304]:
#probit_results = est.estimate(probit.q, theta0, y, x)

In [305]:
#probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')
#probit_tab

In [306]:
theta0 = logit.starting_values(y, x)
theta0 

array([ 1.90417771e-02,  5.02863186e-02,  2.73331524e-02, -7.37894881e-05,
        1.66389576e-02, -4.05348732e-03])

In [307]:
ll = logit.loglikelihood(theta0, y, x)
np.isclose(np.mean(ll),-0.9974267061091704)

np.False_

In [308]:
logit_results = est.estimate(logit.q, theta0, y, x)

Optimization terminated successfully.
         Current function value: 0.033326
         Iterations: 65
         Function evaluations: 574
         Gradient evaluations: 82


In [309]:
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')
logit_tab

Optimizer succeeded after 65 iter. (574 func. evals.). Final criterion:  0.03333.
Logit, y = anyuseofforce_coded


,theta,se,t
sblack,0.1378,0.7223,0.1907
shisp,0.6087,0.5319,1.1443
smale,0.0021,0.6597,0.0032
sage,-0.1628,0.0168,-9.6694
omajhisp,0.3236,1.3030,0.2484
daytime,-0.9576,0.5766,-1.6607


In [312]:
# Let us make a vector of the values we want to investigate
x_me = np.array([1,0,1,25,1,1]).reshape(1, K) # recall, x_i is a *row* vector
pd.DataFrame(x_me, columns=x_lab, index=['x_me']) # print it out 

,sblack,shisp,smale,sage,omajhisp,daytime
x_me,1,0,1,25,1,1


In [353]:
# We will look at the same values as previously, but we want to look at the difference for foreign = 0 and foreign = 1.
k = 0
x_me2 = x_me.copy()
x_me2[:, k] = 1  # Keep everythin the same, but change foreign to 1 for all obs. 

b_lg = logit_tab.theta.values

# For the probit, calculate the norm.cdf for foreign = 1, and subtract foreign = 0
#me_foreign_pr = probit.G(x_me2@b_pr) - probit.G(x_me@b_pr) 

# The same for the logit, calculate using the G() function for logit, for foreign = 1, and subtract foreign = 0
me_sblack_lg = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)

# print results 
pd.DataFrame([ols_results['b_hat'][k], 
              me_sblack_lg[0]],
             index=['OLS', 'Logit'], columns=[f'Marg. Eff.: {x_lab[k]}']).round(4)

,Marg. Eff.: sblack
OLS,0.0048
Logit,0.0000


In [355]:
# We will look at the same values as previously, but we want to look at the difference for foreign = 0 and foreign = 1.
k = 1
x_me2 = x_me.copy()
x_me2[:, k] = 1  # Keep everythin the same, but change foreign to 1 for all obs. 

b_lg = logit_tab.theta.values

# For the probit, calculate the norm.cdf for foreign = 1, and subtract foreign = 0
#me_foreign_pr = probit.G(x_me2@b_pr) - probit.G(x_me@b_pr) 

# The same for the logit, calculate using the G() function for logit, for foreign = 1, and subtract foreign = 0
me_shisp_lg = logit.G(x_me2@b_lg) - logit.G(x_me@b_lg)

# print results 
pd.DataFrame([ols_results['b_hat'][k], 
              me_shisp_lg[0]],
             index=['OLS', 'Logit'], columns=[f'Marg. Eff.: {x_lab[k]}']).round(4)

,Marg. Eff.: shisp
OLS,0.0126
Logit,0.0085


In [349]:
bb = np.outer(b_lg,b_lg)
xx = np.outer(x_me, x_me)
I_K = np.eye(K)
gx0 = norm.pdf(x_me@b_lg)
gx2 = norm.pdf(x_me2@b_lg)

grad_c_lg = gx0*(np.eye(K) - bb @ xx)
grad_d_lg = gx2*x_me2 - gx0*x_me

In [361]:
grad_c_lg.shape == (6,6)

True

In [360]:
grad_d_lg.shape == (1,6)

True

In [350]:
def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt(np.diag(cov_me))

se_c_pr = get_se(grad_c_lg, logit_results['cov'])
se_d_pr = get_se(grad_d_lg, logit_results['cov'])

In [367]:
# extract scalar standard errors (assuming se_d_pr[0] for sblack, se_d_pr[1] for shisp)
se_sblack = float(se_d_pr[0])
se_shisp  = float(se_d_pr[1])

me_dict = {
    'Marginal Effect': [me_sblack, me_shisp],
    's.e.':            [se_sblack, se_shisp],
}
tab = pd.DataFrame(me_dict,index=['sblack', 'shisp'])
tab['t'] = tab['Marginal Effect'] / tab['s.e.']
tab.index.name = 'Var'
tab.round(4)


IndexError: index 1 is out of bounds for axis 0 with size 1